In [1]:
import torch
import torch.nn as nn

In [2]:
from data_preparation import load_data_tensor

lr_train, lr_test, hr_train = load_data_tensor("dgl-icl")


In [3]:
A = torch.randint(0, 3, (2, 2, 2))
B = torch.randint(0, 3, (2, 2, 2))

In [4]:
A[1] @ B[1]

tensor([[2, 0],
        [4, 2]])

In [5]:
B

tensor([[[1, 2],
         [2, 0]],

        [[1, 1],
         [2, 0]]])

In [6]:
torch.bmm(A, B)

tensor([[[2, 4],
         [0, 0]],

        [[2, 0],
         [4, 2]]])

In [7]:
lr_train.shape
lr_stack = lr_train.reshape(-1, 160)
hr_stack = hr_train.reshape(-1,268)
print(lr_stack[0])

tensor([0.0000, 0.3388, 0.2025, 0.6895, 0.6105, 0.4464, 0.6495, 0.6894, 0.4389,
        0.5773, 0.0000, 0.4907, 0.2030, 0.5973, 0.1080, 0.5209, 0.2436, 0.3475,
        0.3142, 0.4680, 0.4328, 0.6836, 0.1751, 0.4156, 0.2886, 0.4879, 0.5107,
        0.0409, 0.6092, 0.3186, 0.3773, 0.5973, 0.4236, 0.5157, 0.0000, 0.2851,
        0.0880, 0.1500, 0.0387, 0.1536, 0.3912, 0.6067, 0.0019, 0.2677, 0.3644,
        0.1659, 0.2804, 0.0000, 0.2246, 0.0000, 0.0168, 0.0154, 0.2328, 0.1669,
        0.3741, 0.0000, 0.0000, 0.3887, 0.5526, 0.0068, 0.5261, 0.0912, 0.3366,
        0.1353, 0.0618, 0.1170, 0.3364, 0.1876, 0.2191, 0.0557, 0.0009, 0.1510,
        0.0694, 0.0450, 0.2801, 0.0509, 0.0000, 0.2265, 0.2669, 0.0807, 0.7052,
        0.1387, 0.1906, 0.3906, 0.0000, 0.2254, 0.4377, 0.2353, 0.0193, 0.2244,
        0.0000, 0.2057, 0.0000, 0.2625, 0.2219, 0.2037, 0.3883, 0.3881, 0.2607,
        0.1453, 0.0633, 0.1122, 0.2182, 0.0832, 0.2598, 0.3783, 0.2654, 0.1461,
        0.1611, 0.0000, 0.2819, 0.2389, 

In [8]:
def standardization(matrix):
    avg = torch.mean(matrix)
    std = torch.std(matrix)
    return (matrix - avg)/std

In [9]:
lr_train2 = torch.bmm(lr_train, lr_train)
lr_stack2 = lr_train2.reshape(-1, 160)
hr_train2 = torch.bmm(hr_train, hr_train)
hr_stack2 = hr_train2.reshape(-1, 268)
print(lr_stack2.shape)

torch.Size([26720, 160])


In [10]:
lr_train2 = torch.bmm(lr_train, lr_train)
lr_stack2 = lr_train2.reshape(-1, 160)
hr_train2 = torch.bmm(hr_train, hr_train)
hr_stack2 = hr_train2.reshape(-1, 268)
print(lr_stack2.shape)

torch.Size([26720, 160])


In [11]:
def adj_hop(adj, hop):
    adj_new = adj
    for i in range(hop-1):
        adj_new = torch.bmm(adj_new, adj)
    stack_adj = adj_new.reshape(-1,adj.shape[1])
    return stack_adj 




In [12]:
print(lr_stack2)
print(adj_hop(lr_train,2))

tensor([[14.3806,  7.2528,  6.9529,  ...,  9.8382,  7.9998,  4.3059],
        [ 7.2528,  6.7310,  5.0556,  ...,  6.8077,  4.0328,  4.0265],
        [ 6.9529,  5.0556,  8.9668,  ...,  7.0718,  5.5102,  2.7241],
        ...,
        [ 5.8574,  6.6959,  5.0896,  ...,  6.7481,  2.8670,  5.2785],
        [ 1.6789,  1.6876,  2.2906,  ...,  2.8670,  7.0863,  4.0678],
        [ 4.7431,  5.3005,  4.3073,  ...,  5.2785,  4.0678,  6.6843]])
tensor([[14.3806,  7.2528,  6.9529,  ...,  9.8382,  7.9998,  4.3059],
        [ 7.2528,  6.7310,  5.0556,  ...,  6.8077,  4.0328,  4.0265],
        [ 6.9529,  5.0556,  8.9668,  ...,  7.0718,  5.5102,  2.7241],
        ...,
        [ 5.8574,  6.6959,  5.0896,  ...,  6.7481,  2.8670,  5.2785],
        [ 1.6789,  1.6876,  2.2906,  ...,  2.8670,  7.0863,  4.0678],
        [ 4.7431,  5.3005,  4.3073,  ...,  5.2785,  4.0678,  6.6843]])


In [13]:
end_dim_1 = 32

class AutoencoderLR(nn.Module):
    def __init__(self):
        super(AutoencoderLR, self).__init__()
        self.encoder = nn.Sequential(nn.Linear(161, 128),
        nn.BatchNorm1d(128),              
        nn.ReLU(),
        nn.Linear(128,64),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.Linear(64,32),
        nn.BatchNorm1d(32),
        #nn.ReLU(),
        #nn.Linear(32, end_dim),
        )
        self.decoder = nn.Sequential(
        #nn.Linear(end_dim, 32),
        #nn.BatchNorm1d(32),
        #nn.ReLU(),
        nn.Linear(32,64),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(64,128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(128,160),
        )
    
    def encode(self,adj_row, hops):
        x = torch.cat([adj_row, hops], dim = 1)
        x = self.encoder(x)
        return x
    def decode(self, latent_input):
        return self.decoder(latent_input)

    def forward(self, adj_row, hoops):
        x = self.encode(adj_row, hoops)
        x = self.decode(x)
        return x


In [14]:
end_dim = 32

class AutoencoderHR(nn.Module):
    def __init__(self):
        super(AutoencoderHR, self).__init__()
        self.encoder = nn.Sequential(nn.Linear(269, 128),
        nn.BatchNorm1d(128),              
        nn.ReLU(),
        nn.Linear(128,64),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.Linear(64,32),
        nn.BatchNorm1d(32),
        #nn.ReLU(),
        #nn.Linear(32, end_dim),
        )
        self.decoder = nn.Sequential(
        #nn.Linear(end_dim, 32),
        #nn.BatchNorm1d(32),
        #nn.ReLU(),
        nn.Linear(32,64),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(64,128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(128,268),
        )
    
    def encode(self,adj_row, hops):
        x = torch.cat([adj_row, hops], dim = 1)
        x = self.encoder(x)
        return x
    def decode(self, latent_input):
        return self.decoder(latent_input)

    def forward(self, adj_row, hoops):
        x = self.encode(adj_row, hoops)
        x = self.decode(x)
        return x

In [90]:
from torch.nn import functional as F

in_dim = 160

class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(in_dim + 1, 128),
            nn.MaxPool1d(2),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64,32),
            nn.BatchNorm1d(32),
            nn.ReLU(),

            nn.Sigmoid()

        )
        self.fc_m = nn.Linear(32,16)
        self.fc_std = nn.Linear(32,16)
        self.decoder = nn.Sequential(
            nn.Linear(16, 32),
            nn.BatchNorm1d(32),
            nn.LeakyReLU(0.2),
            nn.Linear(32,64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(0.2),
            nn.Linear(64,128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, in_dim)
        )
    def encode(self, x, hops):
        x = torch.cat([x, hops], dim = 1)
        h1 = self.encoder(x)
        return self.fc_m(h1), self.fc_std(h1)
    
    def reparametrize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std
    
    def decode(self, z):
        return self.decoder(z)
    
    def forward(self, x, hops):
        mu, logvar = self.encode(x, hops)
        z = self.reparametrize(mu,logvar)
        return self.decode(z), mu, logvar
    
        

In [66]:
from torch.nn import functional as F

in_dim = 268

class VAEHR(nn.Module):
    def __init__(self):
        super(VAEHR, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(in_dim + 1, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.MaxPool1d(2),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64,32),
            nn.BatchNorm1d(32),
            nn.ReLU(),

            nn.Sigmoid()

        )
        self.fc_m = nn.Linear(32,16)
        self.fc_std = nn.Linear(32,16)
        self.decoder = nn.Sequential(
            nn.Linear(16, 32),
            nn.BatchNorm1d(32),
            nn.LeakyReLU(0.2),
            nn.Linear(32,64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(0.2),
            nn.Linear(64,128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, in_dim)
        )
    def encode(self, x, hops):
        x = torch.cat([x, hops], dim = 1)
        h1 = self.encoder(x)
        return self.fc_m(h1), self.fc_std(h1)
    
    def reparametrize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std
    
    def decode(self, z):
        return self.decoder(z)
    
    def forward(self, x, hops):
        mu, logvar = self.encode(x, hops)
        z = self.reparametrize(mu,logvar)
        return self.decode(z), mu, logvar
    
        

In [91]:


def loss_function(recon_x, x, mu, logvar):
    BCE = F.mse_loss(recon_x, x, reduction="sum")
    KLD = -0.5*torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

In [92]:

def train(model, adj_matrix, num_epoch = 100, lr = 0.0001, batch_size = 64):
    optimizer = torch.optim.AdamW(model.parameters(), lr = lr)
    for step in range(num_epoch):
        n_completed = 0
        while n_completed < len(adj_matrix):
            optimizer.zero_grad()
            batch = adj_matrix[n_completed: n_completed+batch_size]
            e, mu, logvar = model(batch, torch.ones(batch.shape[0]).reshape(-1,1)*2)
            loss = loss_function(e,batch, mu, logvar)
            loss.backward()
            print(loss)
            optimizer.step()
            n_completed += batch_size
    

In [93]:
print(lr_stack2)
norm = standardization(lr_stack2)
print(norm)

tensor([[14.3806,  7.2528,  6.9529,  ...,  9.8382,  7.9998,  4.3059],
        [ 7.2528,  6.7310,  5.0556,  ...,  6.8077,  4.0328,  4.0265],
        [ 6.9529,  5.0556,  8.9668,  ...,  7.0718,  5.5102,  2.7241],
        ...,
        [ 5.8574,  6.6959,  5.0896,  ...,  6.7481,  2.8670,  5.2785],
        [ 1.6789,  1.6876,  2.2906,  ...,  2.8670,  7.0863,  4.0678],
        [ 4.7431,  5.3005,  4.3073,  ...,  5.2785,  4.0678,  6.6843]])
tensor([[ 1.2825,  0.0442, -0.0079,  ...,  0.4934,  0.1740, -0.4677],
        [ 0.0442, -0.0464, -0.3375,  ..., -0.0331, -0.5152, -0.5163],
        [-0.0079, -0.3375,  0.3420,  ...,  0.0128, -0.2585, -0.7425],
        ...,
        [-0.1982, -0.0525, -0.3316,  ..., -0.0434, -0.7177, -0.2988],
        [-0.9241, -0.9226, -0.8178,  ..., -0.7177,  0.0153, -0.5091],
        [-0.3918, -0.2949, -0.4675,  ..., -0.2988, -0.5091, -0.0545]])


In [97]:
model = VAE()
params = sum(p.numel() for p in model.parameters() if p.requires_grad_)
print(params)
lr_stack2 = adj_hop(lr_train,2)
norm = standardization(lr_stack2)

train(model, norm)


56128
tensor(5725.9385, grad_fn=<AddBackward0>)
tensor(5795.5088, grad_fn=<AddBackward0>)
tensor(24331.3125, grad_fn=<AddBackward0>)
tensor(23327.0977, grad_fn=<AddBackward0>)
tensor(14342.7197, grad_fn=<AddBackward0>)
tensor(8894.2393, grad_fn=<AddBackward0>)
tensor(5875.4468, grad_fn=<AddBackward0>)
tensor(7664.8960, grad_fn=<AddBackward0>)
tensor(6810.1196, grad_fn=<AddBackward0>)
tensor(6462.7461, grad_fn=<AddBackward0>)
tensor(7174.6016, grad_fn=<AddBackward0>)
tensor(14139.2793, grad_fn=<AddBackward0>)
tensor(12949.6836, grad_fn=<AddBackward0>)
tensor(15337.2920, grad_fn=<AddBackward0>)
tensor(26321.4219, grad_fn=<AddBackward0>)
tensor(5663.3047, grad_fn=<AddBackward0>)
tensor(6081.0679, grad_fn=<AddBackward0>)
tensor(5236.2847, grad_fn=<AddBackward0>)
tensor(6003.7573, grad_fn=<AddBackward0>)
tensor(6738.1250, grad_fn=<AddBackward0>)
tensor(4711.4736, grad_fn=<AddBackward0>)
tensor(5661.4580, grad_fn=<AddBackward0>)
tensor(8599.8223, grad_fn=<AddBackward0>)
tensor(6180.1162, gra

In [96]:
model.eval()
mu, logvar = model.encode(norm, torch.ones(norm.shape[0]).reshape(-1,1)*2)
res_lr_1 = model.reparametrize(mu,logvar).reshape(167, 160, 16)
torch.save(res_lr_1.detach(), "model_autoencoder/encode_lr_2.pt")


In [ ]:
torch.save(model.state_dict(), "model_autoencoder/autoencoder.pth")